In [1]:
import pandas as pd
import numpy as np

In [2]:
movies_df = pd.read_csv(r"C:\Users\ditri\Desktop\Important\Data\Movie Lens Data\ml-latest-small\movies.csv")
ratings_df = pd.read_csv(r"C:\Users\ditri\Desktop\Important\Data\Movie Lens Data\ml-latest-small\ratings.csv")

In [3]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies_df.shape

(9742, 3)

In [5]:
pattern = r"\((\d{4})\)"

movies_df['release_year'] = movies_df['title'].str.extract(pattern)

In [6]:
movies_df = movies_df.dropna(subset = 'release_year')
movies_df['release_year'] = movies_df['release_year'].astype(np.uint16)

In [7]:
movies_df

,movieId,title,genres,release_year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,2017
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,2017
9739,193585,Flint (2017),Drama,2017
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,2018


In [8]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
ratings_df.shape

(100836, 4)

In [10]:
merged_df = ratings_df.merge(movies_df, on = 'movieId', how = 'left').drop(columns = 'timestamp')
merged_df = merged_df.dropna(subset = 'release_year')
merged_df['release_year'] = merged_df['release_year'].astype(np.uint16)
merged_df.head()

,userId,movieId,rating,title,genres,release_year
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance,1995
2,1,6,4.0,Heat (1995),Action|Crime|Thriller,1995
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995
4,1,50,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995


In [11]:
movie_des = ratings_df.groupby('movieId').agg(                #Movie Description
    avg_rating = ('rating', 'mean'),
    no_of_ratings = ('rating', 'count')
).reset_index()

In [12]:
movie_des.shape

(9724, 3)

In [13]:
user_des = ratings_df.groupby('userId').agg(                              #User Description
    movie_count=('movieId', 'count'),  # Count of movieId
    rating_mean=('rating', 'mean')    # Mean of ratings
).reset_index()

In [14]:
user_des.shape

(610, 3)

In [15]:
popular_df = movie_des.merge(movies_df[['movieId', 'title', 'release_year']], on = 'movieId', how = 'left')
popular_df = popular_df.dropna(subset = 'release_year')
popular_df['release_year'] = popular_df['release_year'].astype(np.int16)
popular_df.head()

,movieId,avg_rating,no_of_ratings,title,release_year
0,1,3.920930,215,Toy Story (1995),1995
1,2,3.431818,110,Jumanji (1995),1995
2,3,3.259615,52,Grumpier Old Men (1995),1995
3,4,2.357143,7,Waiting to Exhale (1995),1995
4,5,3.071429,49,Father of the Bride Part II (1995),1995


In [16]:
p_75 = np.percentile(popular_df['no_of_ratings'], 75)
print('75th percentile', p_75)

75th percentile 9.0


In [17]:
C = (ratings_df.groupby('movieId')['rating'].sum().sum())/(ratings_df.shape[0])
print(C)

3.501556983616962


In [18]:
Weighted_mean_numerator = (popular_df['no_of_ratings'] * popular_df['avg_rating']) + (p_75 * C)
Weighted_mean_denominator = popular_df['no_of_ratings'] + p_75
Weighted_mean = Weighted_mean_numerator / Weighted_mean_denominator

In [19]:
popular_df['weighted_rating'] = Weighted_mean

In [20]:
popular_df

,movieId,avg_rating,no_of_ratings,title,release_year,weighted_rating
0,1,3.920930,215,Toy Story (1995),1995,3.904080
1,2,3.431818,110,Jumanji (1995),1995,3.437093
2,3,3.259615,52,Grumpier Old Men (1995),1995,3.295312
3,4,2.357143,7,Waiting to Exhale (1995),1995,3.000876
4,5,3.071429,49,Father of the Bride Part II (1995),1995,3.138173
...,...,...,...,...,...,...
9719,193581,4.000000,1,Black Butler: Book of the Atlantic (2017),2017,3.551401
9720,193583,3.500000,1,No Game No Life: Zero (2017),2017,3.501401
9721,193585,3.500000,1,Flint (2017),2017,3.501401
9722,193587,3.500000,1,Bungo Stray Dogs: Dead Apple (2018),2018,3.501401


In [21]:
popular_df.sort_values(by = 'weighted_rating', ascending = False)[:5]

,movieId,avg_rating,no_of_ratings,title,release_year,weighted_rating
277,318,4.429022,317,"Shawshank Redemption, The (1994)",1994,4.403417
659,858,4.289062,192,"Godfather, The (1972)",1972,4.253801
2224,2959,4.272936,218,Fight Club (1999),1999,4.242352
921,1221,4.259690,129,"Godfather: Part II, The (1974)",1974,4.210246
46,50,4.237745,204,"Usual Suspects, The (1995)",1995,4.206639


In [34]:
def popular_movies(year, df = popular_df) :

    if year.isdigit():
        year = int(year)
        
        if (year >= df['release_year'].min()) & (year <= df['release_year'].max()) :

            index_list = (df['release_year'] - year).abs().sort_values()[:900].index            
        
            df1 = df.loc[index_list]

            return df1.sort_values(by = 'weighted_rating', ascending = False)[:10][['title' , 'weighted_rating']]

        else :
            print (f"Enter Correct Year between {popular_df['release_year'].min()} and {popular_df['release_year'].max()}")

    elif (year == '') :

        return df.sort_values(by = 'weighted_rating', ascending = False)[:10][['title' ,'weighted_rating']]

    else :

        print (f"Input Year in Correct format of a number between {popular_df['release_year'].min()} and {popular_df['release_year'].max()}")    

In [35]:
def movie_table(year):
    
    df2 = popular_movies(year).reset_index(drop = True)
    
    df2['rating'] = df2['weighted_rating'].round(2)
    recommendations = df2.drop(columns = 'weighted_rating')
    
    return recommendations

In [37]:
# Input Year to get the top 10 most popular movies in that year or closest to that year
# In case no input is received then the top 10 all time popular movies are returned

year_i = input(f"Input year between {popular_df['release_year'].min()} and {popular_df['release_year'].max()} : ")

movie_table(year_i)

,title,rating
0,"Three Billboards Outside Ebbing, Missouri (2017)",4.09
1,Logan (2017),4.07
2,Guardians of the Galaxy (2014),3.98
3,Spotlight (2015),3.95
4,Nightcrawler (2014),3.94
5,The Imitation Game (2014),3.94
6,Whiplash (2014),3.94
7,The Martian (2015),3.92
8,Untitled Spider-Man Reboot (2017),3.92
9,Baby Driver (2017),3.92
